## Method


In [6]:
import scipy.integrate as spint

In [119]:
soil_depths = [0.1,0.2,0.4,0.8,1,2,4,8,10,20,40,60]
n = len(soil_depths)
k = 10000 # number of simulations 

### AFX


pargw1   pargw2    parm       parksat     parpa     parpo      parrd	
0.206436 0.600719  13.388162  338.835315  0.051999  2.267469   0.376446

In [109]:
site = 'Nr1'
loc = 'ComoCreek_new'
method = '8'
dat = pd.DataFrame()
dat['po'] = [2.267469]
dat['pa'] = 0.051999
dat['po_actual'] = 0.129
dat['pa_actual'] = 0.356
dat['root_depth'] = 0.376446
dat['soil_depth'] = 20
dat['po_model'] = dat.po * dat.po_actual
dat['pa_model'] = dat.pa * dat.pa_actual
dat['phi_o'] = 0.477
dat['p'] = 4000

### P301

In [116]:
site = 'P301'
loc = 'providence/theo_p301_flux'
method = '8'
dat = pd.DataFrame()
dat['po'] = [2.602553]
dat['pa'] = 0.853939
dat['po_actual'] = 0.204
dat['pa_actual'] = 0.218
dat['root_depth'] = 1.110408
dat['soil_depth'] = 20
dat['po_model'] = dat.po * dat.po_actual
dat['pa_model'] = dat.pa * dat.pa_actual
dat['phi_o'] = 0.435
dat['p'] = 4000

## Start by computing porosity and PAWS

In [117]:
dat['phi_total'] = dat.apply(compute_total_porosity,axis=1)
dat['phi_plant'] = dat.apply(compute_plant_porosity,axis=1)
dat['fc_storage'] = dat.apply(compute_fc_storage,axis=1)
dat['wp_storage'] = dat.apply(compute_wp_storage,axis=1)
dat['PAWS'] = dat.fc_storage - dat.wp_storage
dat['PAWS_percent'] = dat.PAWS/dat.phi_plant
dat['RD_phi_perc'] = dat.phi_plant/dat.root_depth

In [118]:
dat

,po,pa,po_actual,pa_actual,root_depth,soil_depth,po_model,pa_model,phi_o,p,phi_total,phi_plant,fc_storage,wp_storage,PAWS,PAWS_percent,RD_phi_perc
0,2.602553,0.853939,0.204,0.218,1.110408,20,0.530921,0.186159,0.435,4000,8.678286,0.48296,0.002031,0.000002,0.002029,0.004202,0.43494


In [14]:
pas = np.linspace(start=pa_model-(pa_model/2),stop=pa_model+(pa_model/2),num=n)
pos = np.repeat(po_model,n)
pos.shape = pas.shape

In [15]:
combos = []

for i in np.arange(1,n+1):
    combos.append('s%s'%i)

In [16]:
storage = pd.DataFrame({'po':pos,'pa':pas,'combination':combos})

In [17]:
m = []
ksat = []
po = []
pa = [] 
combo = []

for d in drainage.combination:
    for s in storage.combination:
        m.append(round(drainage.loc[drainage.combination==d,'m'].as_matrix()[0],6))
        ksat.append(round(drainage.loc[drainage.combination==d,'ksat'].as_matrix()[0],6))
        po.append(round(storage.loc[storage.combination==s,'po'].as_matrix()[0],6))
        pa.append(round(storage.loc[storage.combination==s,'pa'].as_matrix()[0],6))
        combo.append('%s:%s'%(d,s))
        
tmp = pd.DataFrame({'m':m,'ksat':ksat,'po':po,'pa':pa,'combo':combo})

In [18]:
tmp.head()

,combo,ksat,m,pa,po
0,d1:s1,10.0245,4.9592,0.00356,0.032637
1,d1:s2,10.0245,4.9592,0.00534,0.032637
2,d1:s3,10.0245,4.9592,0.00712,0.032637
3,d1:s4,10.0245,4.9592,0.00890,0.032637
4,d1:s5,10.0245,4.9592,0.01068,0.032637


In [19]:
tmp.tail()

,combo,ksat,m,pa,po
20,d5:s1,30.0735,4.9592,0.00356,0.032637
21,d5:s2,30.0735,4.9592,0.00534,0.032637
22,d5:s3,30.0735,4.9592,0.00712,0.032637
23,d5:s4,30.0735,4.9592,0.00890,0.032637
24,d5:s5,30.0735,4.9592,0.01068,0.032637


In [13]:
# add in the actual varaibles from the definition files
tmp['m_def'] = m_actual
tmp['ksat_def'] = ksat_actual
tmp['po_def'] = po_actual
tmp['pa_def'] = pa_actual
tmp['root_depth'] = root_depth
tmp['soil_depth'] = soil_depth

In [14]:
# convert to model scalars using the actual values from the definition files
tmp['m_scalar'] = tmp.m/tmp.m_def
tmp['ksat_scalar'] = tmp.ksat/tmp.ksat_def
tmp['po_scalar'] = tmp.po/tmp.po_def
tmp['pa_scalar'] = tmp.pa/tmp.pa_def

In [15]:
tmp

,combo,ksat,m,pa,po,m_def,ksat_def,po_def,pa_def,root_depth,soil_depth,m_scalar,ksat_scalar,po_scalar,pa_scalar
0,d1:s1,10.02450,4.9592,0.00356,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,66.830,0.253,0.010
1,d1:s2,10.02450,4.9592,0.00534,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,66.830,0.253,0.015
2,d1:s3,10.02450,4.9592,0.00712,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,66.830,0.253,0.020
3,d1:s4,10.02450,4.9592,0.00890,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,66.830,0.253,0.025
4,d1:s5,10.02450,4.9592,0.01068,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,66.830,0.253,0.030
5,d2:s1,15.03675,4.9592,0.00356,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,100.245,0.253,0.010
6,d2:s2,15.03675,4.9592,0.00534,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,100.245,0.253,0.015
7,d2:s3,15.03675,4.9592,0.00712,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,100.245,0.253,0.020
8,d2:s4,15.03675,4.9592,0.00890,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,100.245,0.253,0.025
9,d2:s5,15.03675,4.9592,0.01068,0.032637,0.4,0.15,0.129,0.356,3.24,7.24,12.398,100.245,0.253,0.030


In [16]:
# save soils data
tmp.to_pickle('./data/soils_%s_smex%s_v3.pcl'%(site,method))

In [17]:
# save the parameters to text files for experiment spinup
combospth = '/RHESSys/%s/clim/smexperiment/smex%s/%s_smex%s_spinup_combos_v3.txt'%(loc,method,site,method)
pospth = '/RHESSys/%s/clim/smexperiment/smex%s/%s_smex%s_spinup_pos_v3.txt'%(loc,method,site,method)
paspth = '/RHESSys/%s/clim/smexperiment/smex%s/%s_smex%s_spinup_pas_v3.txt'%(loc,method,site,method)
mspth = '/RHESSys/%s/clim/smexperiment/smex%s/%s_smex%s_spinup_ms_v3.txt'%(loc,method,site,method)
ksatspth = '/RHESSys/%s/clim/smexperiment/smex%s/%s_smex%s_spinup_ksats_v3.txt'%(loc,method,site,method)

np.savetxt(combospth,tmp.combo,fmt='%s')
np.savetxt(pospth,tmp.po_scalar,fmt='%f')
np.savetxt(paspth,tmp.pa_scalar,fmt='%f')
np.savetxt(mspth,tmp.m_scalar,fmt='%f')
np.savetxt(ksatspth,tmp.ksat_scalar,fmt='%f')